PART I

In [1]:
import numpy as np
from collections import defaultdict

In [2]:
input = "input.txt"
with open(input, 'r') as infile:
    #Today we split into characters
    data = np.array([[int(c) for c in l.strip()] for l in infile.readlines()])

In [3]:
data

array([[4, 3, 1, ..., 4, 1, 2],
       [3, 1, 3, ..., 3, 4, 1],
       [3, 4, 3, ..., 4, 4, 2],
       ...,
       [1, 2, 4, ..., 1, 3, 1],
       [4, 1, 2, ..., 4, 1, 2],
       [4, 4, 1, ..., 4, 3, 1]])

In [4]:
def create_shortest_path_map():
    #Can't think of a better way to enforce int max value right now :D
    shortest_paths = np.full_like(data, 2**31-1)
    shortest_paths[0,0] = 0
    return shortest_paths

In [5]:
#For each possible last three steps, we need to create a separate map of shortest paths
#Because the optimal path could be suboptimal in the middle, but because of the "better history" it ends up being optimal after all
shortest_paths_for_last_three_steps = defaultdict(create_shortest_path_map)

In [6]:
def perform_step_from(i,j,last_three_steps):
    current_direction = last_three_steps[-1]
    if current_direction == 'N':
        possible_directions = 'WE'
        if last_three_steps != ('N',) *3:
            possible_directions += 'N'
    elif current_direction == 'E':
        possible_directions = 'NS'
        if last_three_steps != ('E',)*3:
            possible_directions += 'E'
    elif current_direction == 'S':
        possible_directions = 'EW'
        if last_three_steps != ('S',)*3:
            possible_directions += 'S'
    elif current_direction == 'W':
        possible_directions = 'NS'
        if last_three_steps != ('W',)*3:
            possible_directions += 'W'
    #At the start, there is no current direction but you can move east and south
    else:
        possible_directions = 'SE'
    res = []
    for new_direction in possible_directions:
        if new_direction == 'N':
            new_last_three_steps = last_three_steps[1:] + tuple('N')
            if i-1 >= 0 and (path_length:=shortest_paths_for_last_three_steps[last_three_steps][i,j] + data[i-1,j]) < shortest_paths_for_last_three_steps[new_last_three_steps][i-1,j]:
                shortest_paths_for_last_three_steps[new_last_three_steps][i-1,j] = path_length
                res.append((i-1,j, new_last_three_steps))
        elif new_direction == 'E':
            new_last_three_steps = last_three_steps[1:] + tuple('E')
            if j+1 < len(data[0]) and (path_length:=shortest_paths_for_last_three_steps[last_three_steps][i,j] + data[i,j+1]) < shortest_paths_for_last_three_steps[new_last_three_steps][i,j+1]:
                shortest_paths_for_last_three_steps[new_last_three_steps][i,j+1] = path_length
                res.append((i,j+1, new_last_three_steps))
        elif new_direction == 'S':
            new_last_three_steps = last_three_steps[1:] + tuple('S')
            if i+1 < len(data) and (path_length:=shortest_paths_for_last_three_steps[last_three_steps][i,j] + data[i+1,j]) < shortest_paths_for_last_three_steps[new_last_three_steps][i+1,j]:
                shortest_paths_for_last_three_steps[new_last_three_steps][i+1,j] = path_length
                res.append((i+1,j, new_last_three_steps))
        elif new_direction == 'W':
            new_last_three_steps = last_three_steps[1:] + tuple('W')
            if j-1 >= 0 and (path_length:=shortest_paths_for_last_three_steps[last_three_steps][i,j] + data[i,j-1]) < shortest_paths_for_last_three_steps[new_last_three_steps][i,j-1]:
                shortest_paths_for_last_three_steps[new_last_three_steps][i,j-1] = path_length
                res.append((i,j-1, new_last_three_steps))
    return res

In [7]:
current_configurations = [(0,0,(None, None, None))]
while current_configurations:
    new_current_configurations = []
    for (i,j,last_three_steps) in current_configurations:
        new_current_configurations += perform_step_from(i,j,last_three_steps)
    current_configurations = new_current_configurations

In [8]:
min(shortest_path_map[-1,-1] for shortest_path_map in shortest_paths_for_last_three_steps.values())

814

In [12]:
shortest_paths_for_last_three_steps.keys()

dict_keys([(None, None, None), (None, None, 'S'), (None, None, 'E'), (None, 'S', 'E'), (None, 'S', 'S'), (None, 'E', 'S'), (None, 'E', 'E'), ('S', 'E', 'N'), ('S', 'E', 'S'), ('S', 'E', 'E'), ('S', 'S', 'E'), ('S', 'S', 'S'), ('E', 'S', 'E'), ('E', 'S', 'W'), ('E', 'S', 'S'), ('E', 'E', 'S'), ('E', 'E', 'E'), ('E', 'N', 'W'), ('E', 'N', 'E'), ('E', 'E', 'N'), ('S', 'W', 'N'), ('S', 'W', 'S'), ('S', 'S', 'W'), ('N', 'E', 'S'), ('N', 'E', 'E'), ('E', 'N', 'N'), ('W', 'S', 'E'), ('W', 'S', 'S'), ('S', 'W', 'W'), ('W', 'N', 'E'), ('W', 'N', 'N'), ('N', 'W', 'S'), ('N', 'W', 'W'), ('N', 'W', 'N'), ('N', 'E', 'N'), ('N', 'N', 'W'), ('N', 'N', 'E'), ('W', 'N', 'W'), ('W', 'S', 'W'), ('W', 'W', 'N'), ('W', 'W', 'S'), ('N', 'N', 'N'), ('W', 'W', 'W')])

Not very efficient (runs for about a minute), but I am not in a hurry today ;-)

PART II:

Instead of remembering the past three steps, now have only 6\*4 maps - for each direction the number of still left possible moves in that direction.

In [1]:
import numpy as np
from collections import defaultdict

In [77]:
input = "input.txt"
with open(input, 'r') as infile:
    #Today we split into characters
    data = np.array([[int(c) for c in l.strip()] for l in infile.readlines()])

In [78]:
data

array([[4, 3, 1, ..., 4, 1, 2],
       [3, 1, 3, ..., 3, 4, 1],
       [3, 4, 3, ..., 4, 4, 2],
       ...,
       [1, 2, 4, ..., 1, 3, 1],
       [4, 1, 2, ..., 4, 1, 2],
       [4, 4, 1, ..., 4, 3, 1]])

In [79]:
def create_shortest_path_map():
    #Can't think of a better way to enforce int max value right now :D
    shortest_paths = np.full_like(data, 2**31-1)
    shortest_paths[0,0] = 0
    return shortest_paths

In [80]:
shortest_paths_for_next_step = defaultdict(create_shortest_path_map)

In [81]:
def perform_step_from(i,j,last_steps,n_possible_steps_in_direction):
    current_direction = last_steps[-1]
    if current_direction == 'N':
        possible_directions = 'WE'
        if n_possible_steps_in_direction > 0:
            possible_directions += 'N'
    elif current_direction == 'E':
        possible_directions = 'NS'
        if n_possible_steps_in_direction > 0:
            possible_directions += 'E'
    elif current_direction == 'S':
        possible_directions = 'EW'
        if n_possible_steps_in_direction > 0:
            possible_directions += 'S'
    elif current_direction == 'W':
        possible_directions = 'NS'
        if n_possible_steps_in_direction > 0:
            possible_directions += 'W'
    #At the start, there is no current direction but you can move east and south
    else:
        possible_directions = 'SE'
    res = []
    for new_direction in possible_directions:
        if new_direction == 'N':
            if current_direction == 'N':
                #We are already moving in that direction, so count how often we can still do that
                new_last_steps = last_steps[1:] + tuple('N')
                #Only move one step, since we were already moving in that direction
                if i-1 >= 0 and (path_length:=shortest_paths_for_next_step[('N', n_possible_steps_in_direction)][i,j] + data[i-1,j]) < shortest_paths_for_next_step[('N', n_possible_steps_in_direction-1)][i-1,j]:
                    shortest_paths_for_next_step[('N', n_possible_steps_in_direction-1)][i-1,j] = path_length
                    res.append((i-1,j, new_last_steps, n_possible_steps_in_direction - 1))
            else:
                #We did not yet move north aka we are turning. So we need to make a 4-field leap, if possible.
                if i-4 >= 0:
                    new_last_steps = last_steps[4:] + tuple('N') *4
                    path_length = shortest_paths_for_next_step[(current_direction, n_possible_steps_in_direction)][i,j] + sum(data[i-offset, j] for offset in range(1,5))
                    if path_length < shortest_paths_for_next_step[('N', 6)][i-4,j]:
                        shortest_paths_for_next_step[('N', 6)][i-4,j] = path_length
                        res.append((i-4,j, new_last_steps, 6))
        elif new_direction == 'E':
            if current_direction == 'E':
                #We are already moving in that direction, so count how often we can still do that
                new_last_steps = last_steps[1:] + tuple('E')
                #Only move one step, since we were already moving in that direction
                if j+1 < len(data[0]) and (path_length:=shortest_paths_for_next_step[('E', n_possible_steps_in_direction)][i,j] + data[i,j+1]) < shortest_paths_for_next_step[('E', n_possible_steps_in_direction-1)][i,j+1]:
                    shortest_paths_for_next_step[('E', n_possible_steps_in_direction-1)][i,j+1] = path_length
                    res.append((i,j+1, new_last_steps, n_possible_steps_in_direction - 1))
            else:
                #We did not yet move north aka we are turning. So we need to make a 4-field leap, if possible.
                if j+4 < len(data[0]):
                    new_last_steps = last_steps[4:] + tuple('E')*4
                    path_length = shortest_paths_for_next_step[(current_direction, n_possible_steps_in_direction)][i,j] + sum(data[i, j+offset] for offset in range(1,5))
                    if path_length < shortest_paths_for_next_step[('E', 6)][i,j+4]:
                        shortest_paths_for_next_step[('E', 6)][i,j+4] = path_length
                        res.append((i,j+4, new_last_steps, 6))
        elif new_direction == 'S':
            if current_direction == 'S':
                #We are already moving in that direction, so count how often we can still do that
                new_last_steps = last_steps[1:] + tuple('S')
                #Only move one step, since we were already moving in that direction
                if i+1 < len(data) and (path_length:=shortest_paths_for_next_step[('S', n_possible_steps_in_direction)][i,j] + data[i+1,j]) < shortest_paths_for_next_step[('S', n_possible_steps_in_direction-1)][i+1,j]:
                    shortest_paths_for_next_step[('S', n_possible_steps_in_direction-1)][i+1,j] = path_length
                    res.append((i+1,j, new_last_steps, n_possible_steps_in_direction - 1))
            else:
                #We did not yet move north aka we are turning. So we need to make a 4-field leap, if possible.
                if i+4 < len(data):
                    new_last_steps = last_steps[4:] + tuple('S') * 4
                    path_length = shortest_paths_for_next_step[(current_direction, n_possible_steps_in_direction)][i,j] + sum(data[i+offset, j] for offset in range(1,5))
                    if path_length < shortest_paths_for_next_step[('S', 6)][i+4,j]:
                        shortest_paths_for_next_step[('S', 6)][i+4,j] = path_length
                        res.append((i+4,j, new_last_steps, 6))
        elif new_direction == 'W':
            if current_direction == 'W':
                #We are already moving in that direction, so count how often we can still do that
                new_last_steps = last_steps[1:] + tuple('W')
                #Only move one step, since we were already moving in that direction
                if j-1 >= 0 and (path_length:=shortest_paths_for_next_step[('W', n_possible_steps_in_direction)][i,j] + data[i,j-1]) < shortest_paths_for_next_step[('W', n_possible_steps_in_direction-1)][i,j-1]:
                    shortest_paths_for_next_step[('W', n_possible_steps_in_direction-1)][i,j-1] = path_length
                    res.append((i,j-1, new_last_steps, n_possible_steps_in_direction - 1))
            else:
                #We did not yet move north aka we are turning. So we need to make a 4-field leap, if possible.
                if j-4 >= 0:
                    new_last_steps = last_steps[4:] + tuple('W')*4
                    path_length = shortest_paths_for_next_step[(current_direction, n_possible_steps_in_direction)][i,j] + sum(data[i, j-offset] for offset in range(1,5))
                    if path_length < shortest_paths_for_next_step[('W', 6)][i,j-4]:
                        shortest_paths_for_next_step[('W', 6)][i,j-4] = path_length
                        res.append((i,j-4, new_last_steps, 6))
    return res

In [82]:
current_configurations = [(0,0,(None,)*10,10)]
while current_configurations:
    new_current_configurations = []
    for (i,j,last_steps,n_possible_steps) in current_configurations:
        new_current_configurations += perform_step_from(i,j,last_steps,n_possible_steps)
    current_configurations = new_current_configurations

In [83]:
min(shortest_path_map[-1,-1] for shortest_path_map in shortest_paths_for_next_step.values())

974

Takes around two minutes to execute, so probably there's a more efficient solution.